In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train1 = pd.read_csv('C:/Users/User/Desktop/빅콘테스트 챔피언스리그/2019빅콘테스트_챔피언스리그_데이터_수정/train_activity.csv', engine = 'python')
train2 = pd.read_csv('C:/Users/User/Desktop/빅콘테스트 챔피언스리그/2019빅콘테스트_챔피언스리그_데이터_수정/train_combat.csv', engine = 'python')
train3 = pd.read_csv('C:/Users/User/Desktop/빅콘테스트 챔피언스리그/2019빅콘테스트_챔피언스리그_데이터_수정/train_pledge.csv', engine = 'python')
train4 = pd.read_csv('C:/Users/User/Desktop/빅콘테스트 챔피언스리그/2019빅콘테스트_챔피언스리그_데이터_수정/train_payment.csv', engine = 'python')
train5 = pd.read_csv('C:/Users/User/Desktop/빅콘테스트 챔피언스리그/2019빅콘테스트_챔피언스리그_데이터_수정/train_trade.csv', engine = 'python')
y_train = pd.read_csv('C:/Users/User/Desktop/빅콘테스트 챔피언스리그/2019빅콘테스트_챔피언스리그_데이터_수정/train_label.csv', engine = 'python')

In [3]:
# 잔존 고객 0, 이탈 고객 1
y_train['leave'] = 0
y_train['leave'][y_train.survival_time < 64] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# 결제 기록이 없으면 0, 결제 기록이 있으면 1
y_train['amount'] = 0
y_train['amount'][y_train.amount_spent > 0] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 42)
for train_index, test_index in split.split(y_train, y_train['leave']):
    strat_y_train = y_train.loc[train_index]
    strat_y_test = y_train.loc[test_index]

In [6]:
len(strat_y_train), len(strat_y_test)

(30000, 10000)

In [7]:
strat_y_train['leave'].mean(), strat_y_test['leave'].mean()

(0.4501, 0.4501)

In [8]:
y_train = strat_y_train.sort_values(['acc_id'], ascending = True)
y_test = strat_y_test.sort_values(['acc_id'], ascending = True)

In [9]:
train_id = y_train['acc_id'].values
test_id = y_test['acc_id'].values

In [10]:
pd.DataFrame(y_train['acc_id']).to_csv('train_index.csv', index = False)
pd.DataFrame(y_test['acc_id']).to_csv('test_index.csv', index = False)

# 전처리

In [27]:
def divide_by_week(data, variable, user_id):
    '''
    데이터와 변수를 넣으면 각 유저 아이디에 대한 주차별 변수의 합을 구함
    '''
    
    week_1 = np.zeros(len(user_id))
    week_2 = np.zeros(len(user_id))
    week_3 = np.zeros(len(user_id))
    week_4 = np.zeros(len(user_id))
    for i in range(0, len(user_id)):
        data_sub = data[data['acc_id'] == user_id[i]]
        week_1[i] = data_sub[(data_sub['day'] >= 1) & (data_sub['day'] <= 7)][variable].sum()
        week_2[i] = data_sub[(data_sub['day'] >= 8) & (data_sub['day'] <= 14)][variable].sum()
        week_3[i] = data_sub[(data_sub['day'] >= 15) & (data_sub['day'] <= 21)][variable].sum()
        week_4[i] = data_sub[(data_sub['day'] >= 22) & (data_sub['day'] <= 28)][variable].sum()
    print(variable, 'done')
        
    return week_1, week_2, week_3, week_4

In [38]:
def preprocess(data1, data2, data3, data4, data5, user_id):
    '''
    유저와 케릭터에 대한 28일 동안의 모든 활동 데이터를 각 유저에 대한 정보로 전처리
    data1 : 기본 활동 데이터(DataFrame)
    data2 : 전투 데이터(DataFrame)
    data3 : 혈맹 데이터(DataFrame)
    data4 : 결제 데이터(DataFrame)
    data5 : 거래 데이터(DataFrame)
    user_id(=acc_id) : 유저별 id(np.array)
    '''
    
    # user_id에 해당하는 행만 subset
    index_list = []
    for i in range(0, len(data1)):
        if(data1['acc_id'][i] in user_id):
            index_list.append(i)            
    data1_sub = data1.loc[index_list]
    
    index_list = []
    for i in range(0, len(data2)):
        if(data2['acc_id'][i] in user_id):
            index_list.append(i)            
    data2_sub = data2.loc[index_list]
    
    index_list = []
    for i in range(0, len(data3)):
        if(data3['acc_id'][i] in user_id):
            index_list.append(i)            
    data3_sub = data3.loc[index_list]
    
    index_list = []
    for i in range(0, len(data4)):
        if(data4['acc_id'][i] in user_id):
            index_list.append(i)            
    data4_sub = data4.loc[index_list]
    
    index_list = []
    for i in range(0, len(data5)):
        if(data5['source_acc_id'][i] in user_id):
            index_list.append(i)      
    source_data = data5.loc[index_list]
    
    index_list = []
    for i in range(0, len(data5)):
        if(data5['target_acc_id'][i] in user_id):
            index_list.append(i)      
    target_data = data5.loc[index_list]
    
    
    # 연속형 변수명만 추출
    data1_col = data1.columns[4:]
    data2_col = data2.columns[6:]
    data3_col = data3.columns[5:]
    
    
    # 각 유저에 대한 연속형 변수를 주차별 합으로 새로운 변수 만듦
    dic = {}  # 새 변수를 담기 위한 빈 dictionary

    for var_name in data1_col:
        week_1, week_2, week_3, week_4 = divide_by_week(data = data1_sub, variable = var_name, user_id = user_id)
        key_1 = var_name + '_1'
        key_2 = var_name + '_2'
        key_3 = var_name + '_3'
        key_4 = var_name + '_4'
        dic[key_1] = week_1
        dic[key_2] = week_2
        dic[key_3] = week_3
        dic[key_4] = week_4
        
    for var_name in data2_col:
        week_1, week_2, week_3, week_4 = divide_by_week(data = data2_sub, variable = var_name, user_id = user_id)
        key_1 = 'combat_' + var_name + '_1'
        key_2 = 'combat_' + var_name + '_2'
        key_3 = 'combat_' + var_name + '_3'
        key_4 = 'combat_' + var_name + '_4'
        dic[key_1] = week_1
        dic[key_2] = week_2
        dic[key_3] = week_3
        dic[key_4] = week_4

    for var_name in data3_col:
        week_1, week_2, week_3, week_4 = divide_by_week(data = data3_sub, variable = var_name, user_id = user_id)
        key_1 = var_name + '_1'
        key_2 = var_name + '_2'
        key_3 = var_name + '_3'
        key_4 = var_name + '_4'
        dic[key_1] = week_1
        dic[key_2] = week_2
        dic[key_3] = week_3
        dic[key_4] = week_4
    
    
    # user_id와 연속형 변수들을 concatenate 하여 data_array로 저장
    user_id_clone = user_id.copy()
    data_array = user_id_clone.reshape(-1, 1)

    for key in dic.keys():
        data_array = np.concatenate([data_array, dic[key].reshape(-1, 1)], axis = 1)
        
        
    # data_array를 데이터프레임 형태로 바꿔주고 컬럼명 지정
    colnames = ['acc_id']
    colnames.extend(list(dic.keys())[0:])

    data = pd.DataFrame(data_array)
    data.columns = colnames
    
    
    # 결제 데이터에서 28일 동안 결제 횟수, 1회 결제시 결제 금액 추출    
    amount_cnt = data4_sub.groupby('acc_id')['amount_spent'].count()
    amount_mean = data4_sub.groupby('acc_id')['amount_spent'].sum() / data4_sub.groupby('acc_id')['amount_spent'].count()
    acc_id_sorted = np.sort(data4_sub['acc_id'].unique())
    
    amount_array = np.concatenate([acc_id_sorted.reshape(-1, 1), amount_cnt.values.reshape(-1, 1),
                                  amount_mean.values.reshape(-1, 1)], axis = 1)
    amount_data = pd.DataFrame(amount_array)
    amount_data.columns = ['acc_id', 'amount_cnt', 'amount_mean']
    
    data = pd.merge(data, amount_data, how = 'left', on = 'acc_id')
    data = data.fillna(0)  # 결제 기록이 없어 nan인 값을 모두 0으로 대체
    
    
    # 전투 데이터의 서버, 직업, 레벨 변수를 가장 접속일 수가 많은 케릭터 기준으로 추출    
    char_list = []
    char_matrix = data2_sub.groupby(['acc_id', 'char_id']).count()['day']
    for i in user_id:
        char_list.append(np.argmax(char_matrix[i]))
        
    server_list = []
    class_list = []
    level_list = []
    for i in char_list:
        user_char = data2_sub[data2_sub['char_id'] == i]
        char_server = user_char['server'].values[-1]
        char_class = user_char['class'].values[-1]
        char_level = user_char['level'].values[-1]
        server_list.append(char_server)
        class_list.append(char_class)
        level_list.append(char_level)
    
    char_array = np.concatenate([np.array(user_id).reshape(-1, 1), np.array(server_list).reshape(-1, 1),
                            np.array(class_list).reshape(-1, 1), np.array(level_list).reshape(-1, 1)], axis = 1)
    char_data = pd.DataFrame(char_array)
    char_data.columns = ['acc_id', 'server', 'class', 'level']
    char_data['acc_id'] = char_data['acc_id'].astype('float64')
    data = pd.merge(char_data, data, how = 'left', on = 'acc_id')
    
    
    # 주차별 접속 시간 변화율 파생 변수
    data['playtime_rate_1'] = (data['playtime_2'] + 1) / (data['playtime_1'] + 1)
    data['playtime_rate_2'] = (data['playtime_3'] + 1) / (data['playtime_2'] + 1)
    data['playtime_rate_3'] = (data['playtime_4'] + 1) / (data['playtime_3'] + 1)
    
    
    # 범주형 변수 one-hot encoding
    server_dummies = pd.get_dummies(data['server']).rename(columns = lambda x : 'server_' + str(x))
    class_dummies = pd.get_dummies(data['class']).rename(columns = lambda x : 'class_' + str(x))

    data = pd.concat([data, server_dummies, class_dummies], axis = 1)
    data.drop(['server', 'class'], axis = 1, inplace = True)
    
    
    # 거래 데이터를 user_id에 대하여 판매, 구매로 구분
    trade_data = pd.DataFrame(user_id)
    trade_data.columns = ['acc_id']
    
    source_data_type_0 = source_data[source_data['type'] == 0]
    source_data_type_1 = source_data[source_data['type'] == 1]
    
    for i in range(1, 5):
        source_data_type_0_week = source_data_type_0[(source_data_type_0['day'] >= (7 * i - 6)) & (source_data_type_0['day'] <= (7 * i))]
        source_data_type_0_week_sum = source_data_type_0_week.groupby(['source_acc_id', 'item_type'])['item_price'].sum().unstack('item_type')
        source_data_type_0_week_sum = source_data_type_0_week_sum.fillna(0)
        source_data_type_0_week_sum = source_data_type_0_week_sum.reset_index(level = 'source_acc_id')
        source_data_type_0_week_sum = source_data_type_0_week_sum.rename(columns = {'source_acc_id' : 'acc_id'})
        col = 'source_type_0_week'+ i + '_' + source_data_type_0_week_sum.columns[1:]
        source_data_type_0_week_sum.columns = source_data_type_0_week_sum.columns[0:1].append(col)
        trade_data = pd.merge(trade_data, source_data_type_0_week_sum, how = 'left', on = 'acc_id')
    
    for i in range(1, 5):
        source_data_type_1_week = source_data_type_1[(source_data_type_1['day'] >= (7 * i - 6)) & (source_data_type_1['day'] <= (7 * i))]
        source_data_type_1_week_sum = source_data_type_1_week.groupby(['source_acc_id', 'item_type'])['item_amount'].sum().unstack('item_type')
        source_data_type_1_week_sum = source_data_type_1_week_sum.fillna(0)
        source_data_type_1_week_sum = source_data_type_1_week_sum.reset_index(level = 'source_acc_id')
        source_data_type_1_week_sum = source_data_type_1_week_sum.rename(columns = {'source_acc_id' : 'acc_id'})
        col = 'source_type_1_week'+ i + '_' + source_data_type_1_week_sum.columns[1:]
        source_data_type_1_week_sum.columns = source_data_type_1_week_sum.columns[0:1].append(col)
        trade_data = pd.merge(trade_data, source_data_type_1_week_sum, how = 'left', on = 'acc_id')
        
    target_data_type_0 = target_data[target_data['type'] == 0]
    target_data_type_1 = target_data[target_data['type'] == 1]
    
    for i in range(1, 5):
        target_data_type_0_week = target_data_type_0[(target_data_type_0['day'] >= (7 * i - 6)) & (target_data_type_0['day'] <= (7 * i))]
        target_data_type_0_week_sum = target_data_type_0_week.groupby(['target_acc_id', 'item_type'])['item_price'].sum().unstack('item_type')
        target_data_type_0_week_sum = target_data_type_0_week_sum.fillna(0)
        target_data_type_0_week_sum = target_data_type_0_week_sum.reset_index(level = 'target_acc_id')
        target_data_type_0_week_sum = target_data_type_0_week_sum.rename(columns = {'target_acc_id' : 'acc_id'})
        col = 'target_type_0_week'+ i + '_' + target_data_type_0_week_sum.columns[1:]
        target_data_type_0_week_sum.columns = target_data_type_0_week_sum.columns[0:1].append(col)
        trade_data = pd.merge(trade_data, target_data_type_0_week_sum, how = 'left', on = 'acc_id')
        
    for i in range(1, 5):
        target_data_type_1_week = target_data_type_1[(target_data_type_1['day'] >= (7 * i - 6)) & (target_data_type_1['day'] <= (7 * i))]
        target_data_type_1_week_sum = target_data_type_1_week.groupby(['target_acc_id', 'item_type'])['item_amount'].sum().unstack('item_type')
        target_data_type_1_week_sum = target_data_type_1_week_sum.fillna(0)
        target_data_type_1_week_sum = target_data_type_1_week_sum.reset_index(level = 'target_acc_id')
        target_data_type_1_week_sum = target_data_type_1_week_sum.rename(columns = {'target_acc_id' : 'acc_id'})
        col = 'target_type_1_week'+ i + '_' + target_data_type_1_week_sum.columns[1:]
        target_data_type_1_week_sum.columns = target_data_type_1_week_sum.columns[0:1].append(col)
        trade_data = pd.merge(trade_data, target_data_type_1_week_sum, how = 'left', on = 'acc_id')
        
    trade_data = trade_data.fillna(0)
    
    
    # 주요 거래 아이템(아데나, 강화 주문서, 기타) 외 아이템은 거래 여부에 따라 0, 1로 코딩
    main_item = ['adena', 'enchant_scroll', 'etc']
    item_list = []
     
    for item in main_item:
        for i in range(0, len(trade_data.columns)):
            if(item in trade_data.columns[i]):
                item_list.append(i)                
    trade_data_sub = trade_data.iloc[:, item_list]
    
    weapon_list = []
    for i in range(0, len(trade_data.columns)):
        if('weapon' in trade_data.columns[i]):
            weapon_list.append(i)       
    trade_data_sub['source_weapon'] = 0
    trade_data_sub['source_weapon'][trade_data.iloc[:, weapon_list].iloc[:, :8].sum(axis = 1) > 0] = 1
    trade_data_sub['target_weapon'] = 0
    trade_data_sub['target_weapon'][trade_data.iloc[:, weapon_list].iloc[:, 8:].sum(axis = 1) > 0] = 1
    
    armor_list = []
    for i in range(0, len(trade_data.columns)):
        if('armor' in trade_data.columns[i]):
            armor_list.append(i)     
    trade_data_sub['source_armor'] = 0
    trade_data_sub['source_armor'][trade_data.iloc[:, armor_list].iloc[:, :8].sum(axis = 1) > 0] = 1
    trade_data_sub['target_armor'] = 0
    trade_data_sub['target_armor'][trade_data.iloc[:, armor_list].iloc[:, 8:].sum(axis = 1) > 0] = 1
    
    accessory_list = []
    for i in range(0, len(trade_data.columns)):
        if('accessory' in trade_data.columns[i]):
            accessory_list.append(i)    
    trade_data_sub['source_accessory'] = 0
    trade_data_sub['source_accessory'][trade_data.iloc[:, accessory_list].iloc[:, :8].sum(axis = 1) > 0] = 1
    trade_data_sub['target_accessory'] = 0
    trade_data_sub['target_accessory'][trade_data.iloc[:, accessory_list].iloc[:, 8:].sum(axis = 1) > 0] = 1
    
    spell_list = []
    for i in range(0, len(trade_data.columns)):
        if('spell' in trade_data.columns[i]):
            spell_list.append(i)     
    trade_data_sub['source_spell'] = 0
    trade_data_sub['source_spell'][trade_data.iloc[:, spell_list].iloc[:, :8].sum(axis = 1) > 0] = 1
    trade_data_sub['target_spell'] = 0
    trade_data_sub['target_spell'][trade_data.iloc[:, spell_list].iloc[:, 8:].sum(axis = 1) > 0] = 1
    
    trade_data_sub['acc_id'] = trade_data['acc_id']
    
    data = pd.merge(data, trade_data_sub, how = 'left', on = 'acc_id')
        
    
    return data

In [ ]:
X_train = preprocess(train1, train2, train3, train4, train5, train_id)

In [ ]:
X_train.head()

In [ ]:
X_train.columns.values

In [39]:
X_test = preprocess(train1, train2, train3, train4, train5, test_id)

playtime done
npc_kill done
solo_exp done
party_exp done
quest_exp done
rich_monster done
death done
revive done
exp_recovery done
fishing done
private_shop done
game_money_change done
enchant_count done
pledge_cnt done
random_attacker_cnt done
random_defender_cnt done
temp_cnt done
same_pledge_cnt done
etc_cnt done
num_opponent done
play_char_cnt done
combat_char_cnt done
pledge_combat_cnt done
random_attacker_cnt done
random_defender_cnt done
same_pledge_cnt done
temp_cnt done
etc_cnt done
combat_play_time done
non_combat_play_time done


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:52: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


In [40]:
X_test.head()

,acc_id,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,...,server_bp,server_br,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7
0,20.0,17,5.526287,7.014944,3.508642,9.538170,27.285960,1.631575,1.581222,3.601428,...,0,0,0,0,0,0,1,0,0,0
1,62.0,16,8.983435,11.438782,7.651602,9.952466,27.273118,0.949612,0.734682,1.747826,...,0,0,0,0,0,1,0,0,0,0
2,81.0,16,7.007922,12.358660,11.382606,10.820849,29.873229,4.915677,3.930919,4.844371,...,0,0,0,1,0,0,0,0,0,0
3,86.0,0,22.800322,22.971190,22.510081,22.964168,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,1
4,125.0,16,0.000000,0.414296,0.154483,0.482175,0.000000,0.088540,0.023994,0.487648,...,0,0,0,0,0,0,0,0,0,1


In [43]:
X_test.columns.values

array(['acc_id', 'level', 'playtime_1', 'playtime_2', 'playtime_3',
       'playtime_4', 'npc_kill_1', 'npc_kill_2', 'npc_kill_3',
       'npc_kill_4', 'solo_exp_1', 'solo_exp_2', 'solo_exp_3',
       'solo_exp_4', 'party_exp_1', 'party_exp_2', 'party_exp_3',
       'party_exp_4', 'quest_exp_1', 'quest_exp_2', 'quest_exp_3',
       'quest_exp_4', 'rich_monster_1', 'rich_monster_2',
       'rich_monster_3', 'rich_monster_4', 'death_1', 'death_2',
       'death_3', 'death_4', 'revive_1', 'revive_2', 'revive_3',
       'revive_4', 'exp_recovery_1', 'exp_recovery_2', 'exp_recovery_3',
       'exp_recovery_4', 'fishing_1', 'fishing_2', 'fishing_3',
       'fishing_4', 'private_shop_1', 'private_shop_2', 'private_shop_3',
       'private_shop_4', 'game_money_change_1', 'game_money_change_2',
       'game_money_change_3', 'game_money_change_4', 'enchant_count_1',
       'enchant_count_2', 'enchant_count_3', 'enchant_count_4',
       'combat_pledge_cnt_1', 'combat_pledge_cnt_2',
       'combat

In [64]:
X_train.to_csv('X_train.csv', index = False)
X_test.to_csv('X_test.csv', index = False)

# 모델링

In [2]:
X_train = pd.read_csv('X_train.csv')

In [3]:
X_train.head()

,acc_id,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,solo_exp_1,...,combat_play_time_3,combat_play_time_4,non_combat_play_time_1,non_combat_play_time_2,non_combat_play_time_3,non_combat_play_time_4,survival_time,amount_spent,leave,amount
0,2.0,22.919696,22.980553,22.659883,22.973531,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,64,0.000000,0,0
1,5.0,0.000000,0.589845,0.213000,5.313287,0.000000,0.310229,0.162887,2.763336,0.000000,...,0.593982,3.661882,0.000000,0.000000,0.000000,0.004402,60,0.000000,1,0
2,8.0,4.987936,4.889629,4.653223,4.267014,26.808113,1.325063,1.444694,1.594739,28.720862,...,33.515247,29.782361,0.021776,0.000347,0.010656,0.001042,64,0.020310,0,1
3,17.0,5.406914,9.048973,7.831832,6.176989,25.931158,1.328780,1.362575,1.834339,4.197880,...,14.629460,15.140507,0.000000,0.000000,0.000000,0.000000,64,0.070642,0,1
4,21.0,6.048253,7.363701,4.966870,3.452466,4.379366,4.602406,0.654252,1.135141,14.917062,...,0.149190,0.087337,0.079228,0.408073,0.218226,0.169577,64,0.184267,0,1


In [ ]:
X_train_columns = X_train.columns

In [4]:
from sklearn.preprocessing import MinMaxScaler

minMaxScaler = MinMaxScaler()
X_train = minMaxScaler.fit_transform(X_train.iloc[:, 1:])
X_train

array([[9.88791275e-01, 9.88422430e-01, 9.82742869e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.53699789e-02, 9.23764085e-03, ...,
        0.00000000e+00, 0.00000000e+00, 7.67644201e-05],
       [2.15187317e-01, 2.10309071e-01, 2.01806923e-01, ...,
        1.11180290e-05, 2.83215481e-04, 1.81810469e-05],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.75320610e-01, 7.05325682e-01, 2.62206883e-01, ...,
        7.41201933e-06, 9.23528742e-06, 4.10891659e-03],
       [2.32353832e-01, 2.03161180e-01, 3.19459953e-01, ...,
        3.29834860e-04, 1.23445008e-03, 2.68675470e-04]])

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state = 42)

param_grid = [
    {'penalty' : ['l1', 'l2'], 'C' : [1, 10, 15]}
  ]

grid_search = GridSearchCV(lr, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)
grid_search.fit(X_train, y_train_leave)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  2.4min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'penalty': ['l1', 'l2'], 'C': [1, 10, 15]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=2)

In [30]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7129333333333333
{'C': 10, 'penalty': 'l2'}


In [31]:
lr_clf = LogisticRegression(random_state = 42, C = 10, penalty = 'l2')
lr_clf.fit(X_train, y_train_leave)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
from sklearn.svm import SVC

svc = SVC(random_state=42)

param_grid = [
    {'kernel' : ['linear'], 'C' : [1., 10., 30.]},
    {'kernel' : ['poly'], 'degree' : [2., 3.]},
    {'kernel' : ['rbf'], 'C' : [1., 10., 30.], 'gamma' : [0.1, 1., 3.]},
  ]

grid_search = GridSearchCV(svc, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)
grid_search.fit(X_train, y_train_leave)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 31.3min
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed: 89.3min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'kernel': ['linear'], 'C': [1.0, 10.0, 30.0]}, {'kernel': ['poly'], 'degree': [2.0, 3.0, 4.0]}, {'kernel': ['rbf'], 'C': [1.0, 10.0, 30.0], 'gamma': [0.1, 1.0, 3.0]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=2)

In [20]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7376
{'C': 1.0, 'gamma': 3.0, 'kernel': 'rbf'}


In [32]:
svc_clf = SVC(random_state = 42, C = 1, gamma = 3, kernel = 'rbf', probability = True)
svc_clf.fit(X_train, y_train_leave)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=3, kernel='rbf',
  max_iter=-1, probability=True, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [25]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)

param_grid = [
    {'n_estimators' : [100, 300, 500], 'max_features' : [9, 10, 11, 12]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)
grid_search.fit(X_train, y_train_leave)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 11.2min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [100, 300, 500], 'max_features': [9, 10, 11, 12]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=2)

In [26]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7714
{'max_features': 10, 'n_estimators': 300}


In [33]:
rf_clf = RandomForestClassifier(random_state = 42, max_features = 10, n_estimators = 300)
rf_clf.fit(X_train, y_train_leave)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [35]:
Importance = pd.DataFrame({'feature' : data.columns[1:121], 'importance' : rf_clf.feature_importances_}).sort_values(by = 'importance', ascending = True)
Importance

,feature,importance
48,enchant_count_1,0.000055
49,enchant_count_2,0.000106
51,enchant_count_4,0.000159
50,enchant_count_3,0.000210
70,combat_same_pledge_cnt_3,0.000429
71,combat_same_pledge_cnt_4,0.000513
56,combat_random_attacker_cnt_1,0.000554
69,combat_same_pledge_cnt_2,0.000557
57,combat_random_attacker_cnt_2,0.000557
58,combat_random_attacker_cnt_3,0.000573


In [27]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=42)

param_grid = [
    {'eta' : [0.0001, 0.001, 0.01, 0.1], 'max_depth' : [4, 5, 6, 7], 'gamma' : [0, 1, 10]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)

grid_search.fit(X_train, y_train_leave)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 22.8min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'eta': [0.0001, 0.001, 0.01, 0.1], 'max_depth': [4, 5, 6, 7], 'gamma': [0, 1, 10]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=2)

In [28]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7584
{'eta': 0.0001, 'gamma': 1, 'max_depth': 7}


In [36]:
xgb_clf = XGBClassifier(random_state = 42, eta = 0.0001, gamma = 1, max_depth = 7)
xgb_clf.fit(X_train, y_train_leave)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=0.0001, gamma=1, learning_rate=0.1,
       max_delta_step=0, max_depth=7, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [37]:
Importance = pd.DataFrame({'feature' : data.columns[1:121], 'importance' : xgb_clf.feature_importances_}).sort_values(by = 'importance', ascending = True)
Importance

,feature,importance
48,enchant_count_1,0.000000
49,enchant_count_2,0.002295
70,combat_same_pledge_cnt_3,0.003050
27,death_4,0.003308
101,same_pledge_cnt_2,0.003321
51,enchant_count_4,0.003370
58,combat_random_attacker_cnt_3,0.003612
88,pledge_combat_cnt_1,0.003630
52,combat_pledge_cnt_1,0.003927
56,combat_random_attacker_cnt_1,0.003987


In [38]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

voting_clf = VotingClassifier(
    estimators=[('lr', lr_clf), ('svc', svc_clf), ('rf', rf_clf), ('xgb', xgb_clf)],
    voting='soft')

cv_value = cross_val_score(voting_clf, X_train,
                           y_train_leave, cv = 5,
                           scoring = "accuracy")
np.mean(cv_value)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

0.7543341111935401

In [8]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model

input = Input(shape = (X_train.shape[1], ))

fc1 = Dense(1024, activation='relu')(input)
fc2 = Dense(512, activation='relu')(fc1)
fc3 = Dense(256, activation='relu')(fc2)
fc4 = Dense(128, activation='relu')(fc3)
fc4_drop = Dropout(0.1)(fc4)
fc5 = Dense(64, activation='relu')(fc4_drop)
fc5_drop = Dropout(0.1)(fc5)
fc6 = Dense(32, activation='relu')(fc5_drop)

leave_out = Dense(1, activation='sigmoid', name='leave_output')(fc6)
survival_time_out = Dense(1, activation='linear', name='survival_time_output')(fc6)

model = Model(inputs = input, outputs = [leave_out, survival_time_out])
model.summary()

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 120)          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1024)         123904      input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 512)          524800      dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 256)          131328      dense_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (D

In [10]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model.compile(optimizer = 'rmsprop', loss = {'leave_output' : 'binary_crossentropy', 'survival_time_output' : 'mse'},
              metrics = {'leave_output' : 'acc', 'survival_time_output' : 'mse'})

callbacks_list = [EarlyStopping(monitor = 'val_loss', patience = 20),
                 ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 10)]

history = model.fit(X_train, [y_train_leave, y_train_survival_time],
                    epochs = 150, batch_size = 128, 
                    callbacks = callbacks_list,
                    validation_split = 0.2,
                    shuffle = True)

Train on 24000 samples, validate on 6000 samples
Epoch 1/150
24000/24000 [==============================] - 9s 367us/step - loss: 547.3366 - leave_output_loss: 2.5673 - survival_time_output_loss: 544.7694 - leave_output_acc: 0.4921 - survival_time_output_mean_squared_error: 544.7694 - val_loss: 418.2221 - val_leave_output_loss: 0.7971 - val_survival_time_output_loss: 417.4250 - val_leave_output_acc: 0.5488 - val_survival_time_output_mean_squared_error: 417.4250
Epoch 2/150
24000/24000 [==============================] - 2s 72us/step - loss: 451.7793 - leave_output_loss: 1.0357 - survival_time_output_loss: 450.7435 - leave_output_acc: 0.5149 - survival_time_output_mean_squared_error: 450.7435 - val_loss: 416.0978 - val_leave_output_loss: 0.8286 - val_survival_time_output_loss: 415.2691 - val_leave_output_acc: 0.5488 - val_survival_time_output_mean_squared_error: 415.2691
Epoch 3/150
24000/24000 [==============================] - 2s 72us/step - loss: 427.4786 - leave_output_loss: 0.8459 -

Epoch 40/150
24000/24000 [==============================] - 2s 88us/step - loss: 228.8903 - leave_output_loss: 0.4588 - survival_time_output_loss: 228.4315 - leave_output_acc: 0.7943 - survival_time_output_mean_squared_error: 228.4315 - val_loss: 401.5967 - val_leave_output_loss: 0.5877 - val_survival_time_output_loss: 401.0090 - val_leave_output_acc: 0.7277 - val_survival_time_output_mean_squared_error: 401.0090
Epoch 41/150
24000/24000 [==============================] - 2s 91us/step - loss: 227.0796 - leave_output_loss: 0.4567 - survival_time_output_loss: 226.6230 - leave_output_acc: 0.7948 - survival_time_output_mean_squared_error: 226.6230 - val_loss: 427.9185 - val_leave_output_loss: 0.6070 - val_survival_time_output_loss: 427.3115 - val_leave_output_acc: 0.7255 - val_survival_time_output_mean_squared_error: 427.3115
Epoch 42/150
24000/24000 [==============================] - 2s 88us/step - loss: 225.1292 - leave_output_loss: 0.4546 - survival_time_output_loss: 224.6746 - leave_ou

In [12]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model

input = Input(shape = (X_train.shape[1], ))

fc1 = Dense(1024, activation='relu')(input)
fc2 = Dense(512, activation='relu')(fc1)
fc3 = Dense(256, activation='relu')(fc2)
fc4 = Dense(128, activation='relu')(fc3)
fc4_drop = Dropout(0.1)(fc4)
fc5 = Dense(64, activation='relu')(fc4_drop)
fc5_drop = Dropout(0.1)(fc5)
fc6 = Dense(32, activation='relu')(fc5_drop)

leave_out = Dense(1, activation='sigmoid', name='leave_output')(fc6)
amount_out = Dense(1, activation='sigmoid', name='amount_output')(fc6)

model = Model(inputs = input, outputs = [leave_out, amount_out])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 120)          0                                            
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 1024)         123904      input_2[0][0]                    
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 512)          524800      dense_7[0][0]                    
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 256)          131328      dense_8[0][0]                    
__________________________________________________________________________________________________
dense_10 (

In [13]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model.compile(optimizer = 'rmsprop', loss = {'leave_output' : 'binary_crossentropy', 'amount_output' : 'binary_crossentropy'},
              metrics = {'leave_output' : 'acc', 'amount_output' : 'acc'})

callbacks_list = [EarlyStopping(monitor = 'val_loss', patience = 20),
                 ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 10)]

history = model.fit(X_train, [y_train_leave, y_train_amount],
                    epochs = 150, batch_size = 128, 
                    callbacks = callbacks_list,
                    validation_split = 0.2,
                    shuffle = True)

Train on 24000 samples, validate on 6000 samples
Epoch 1/150
24000/24000 [==============================] - 3s 109us/step - loss: 1.0796 - leave_output_loss: 0.5846 - amount_output_loss: 0.4950 - leave_output_acc: 0.6875 - amount_output_acc: 0.7625 - val_loss: 1.0336 - val_leave_output_loss: 0.5545 - val_amount_output_loss: 0.4792 - val_leave_output_acc: 0.7132 - val_amount_output_acc: 0.7722
Epoch 2/150
24000/24000 [==============================] - 2s 75us/step - loss: 1.0086 - leave_output_loss: 0.5531 - amount_output_loss: 0.4555 - leave_output_acc: 0.7167 - amount_output_acc: 0.7924 - val_loss: 1.0167 - val_leave_output_loss: 0.5463 - val_amount_output_loss: 0.4704 - val_leave_output_acc: 0.7228 - val_amount_output_acc: 0.7883
Epoch 3/150
24000/24000 [==============================] - 2s 76us/step - loss: 0.9871 - leave_output_loss: 0.5424 - amount_output_loss: 0.4447 - leave_output_acc: 0.7215 - amount_output_acc: 0.7982 - val_loss: 1.0553 - val_leave_output_loss: 0.5660 - val_am

# data2(연속형 변수 + 범주형 변수)

In [65]:
train = pd.read_csv('data2.csv')

In [66]:
train.head()

,acc_id,server,class,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,...,non_combat_play_time_1,non_combat_play_time_2,non_combat_play_time_3,non_combat_play_time_4,amount_cnt,amount_mean,survival_time,amount_spent,leave,amount
0,2.0,au,1,0,22.919696,22.980553,22.659883,22.973531,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,64,0.000000,0,0
1,5.0,aq,2,15,0.000000,0.589845,0.213000,5.313287,0.000000,0.310229,...,0.000000,0.000000,0.000000,0.004402,0.0,0.000000,60,0.000000,1,0
2,8.0,ao,4,16,4.987936,4.889629,4.653223,4.267014,26.808113,1.325063,...,0.021776,0.000347,0.010656,0.001042,2.0,0.702322,64,0.020310,0,1
3,17.0,aq,1,16,5.406914,9.048973,7.831832,6.176989,25.931158,1.328780,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,64,0.070642,0,1
4,21.0,ay,0,16,6.048253,7.363701,4.966870,3.452466,4.379366,4.602406,...,0.079228,0.408073,0.218226,0.169577,2.0,1.347730,64,0.184267,0,1


feature                
- 범주형 : 'server', 'class'             
- 연속형 : 그 외('acc_id' 제외)     

target                    
- 잔존 여부 : 'leave', 'survival_time'                     
- 결제 여부 : 'amount', 'amount_spent'

In [67]:
X_train = train.iloc[:, 0:126]
X_train.head()

,acc_id,server,class,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,...,combat_play_time_1,combat_play_time_2,combat_play_time_3,combat_play_time_4,non_combat_play_time_1,non_combat_play_time_2,non_combat_play_time_3,non_combat_play_time_4,amount_cnt,amount_mean
0,2.0,au,1,0,22.919696,22.980553,22.659883,22.973531,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,5.0,aq,2,15,0.000000,0.589845,0.213000,5.313287,0.000000,0.310229,...,0.000000,0.841744,0.593982,3.661882,0.000000,0.000000,0.000000,0.004402,0.0,0.000000
2,8.0,ao,4,16,4.987936,4.889629,4.653223,4.267014,26.808113,1.325063,...,41.123841,32.458405,33.515247,29.782361,0.021776,0.000347,0.010656,0.001042,2.0,0.702322
3,17.0,aq,1,16,5.406914,9.048973,7.831832,6.176989,25.931158,1.328780,...,18.502734,17.420872,14.629460,15.140507,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,21.0,ay,0,16,6.048253,7.363701,4.966870,3.452466,4.379366,4.602406,...,0.228187,0.037182,0.149190,0.087337,0.079228,0.408073,0.218226,0.169577,2.0,1.347730


In [68]:
X_train['playtime_rate_1'] = (X_train['playtime_2'] + 1) / (X_train['playtime_1'] + 1)
X_train['playtime_rate_2'] = (X_train['playtime_3'] + 1) / (X_train['playtime_2'] + 1)
X_train['playtime_rate_3'] = (X_train['playtime_4'] + 1) / (X_train['playtime_3'] + 1)

In [69]:
server_dummies = pd.get_dummies(X_train['server']).rename(columns = lambda x : 'server_' + str(x))
class_dummies = pd.get_dummies(X_train['class']).rename(columns = lambda x : 'class_' + str(x))

X_train = pd.concat([X_train, server_dummies, class_dummies], axis = 1)

In [70]:
X_train.drop(['server', 'class'], axis = 1, inplace = True)
X_train.head()

,acc_id,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,...,server_bp,server_br,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7
0,2.0,0,22.919696,22.980553,22.659883,22.973531,0.000000,0.000000,0.000000,0.000000,...,0,0,0,1,0,0,0,0,0,0
1,5.0,15,0.000000,0.589845,0.213000,5.313287,0.000000,0.310229,0.162887,2.763336,...,0,0,0,0,1,0,0,0,0,0
2,8.0,16,4.987936,4.889629,4.653223,4.267014,26.808113,1.325063,1.444694,1.594739,...,0,0,0,0,0,0,1,0,0,0
3,17.0,16,5.406914,9.048973,7.831832,6.176989,25.931158,1.328780,1.362575,1.834339,...,0,0,0,1,0,0,0,0,0,0
4,21.0,16,6.048253,7.363701,4.966870,3.452466,4.379366,4.602406,0.654252,1.135141,...,0,0,1,0,0,0,0,0,0,0


In [80]:
trade_data_2['acc_id'] = trade_data['acc_id']

In [81]:
X_train = pd.merge(X_train, trade_data_2, how = 'left', on = 'acc_id')

In [82]:
X_train.columns.values

array(['acc_id', 'level', 'playtime_1', 'playtime_2', 'playtime_3',
       'playtime_4', 'npc_kill_1', 'npc_kill_2', 'npc_kill_3',
       'npc_kill_4', 'solo_exp_1', 'solo_exp_2', 'solo_exp_3',
       'solo_exp_4', 'party_exp_1', 'party_exp_2', 'party_exp_3',
       'party_exp_4', 'quest_exp_1', 'quest_exp_2', 'quest_exp_3',
       'quest_exp_4', 'rich_monster_1', 'rich_monster_2',
       'rich_monster_3', 'rich_monster_4', 'death_1', 'death_2',
       'death_3', 'death_4', 'revive_1', 'revive_2', 'revive_3',
       'revive_4', 'exp_recovery_1', 'exp_recovery_2', 'exp_recovery_3',
       'exp_recovery_4', 'fishing_1', 'fishing_2', 'fishing_3',
       'fishing_4', 'private_shop_1', 'private_shop_2', 'private_shop_3',
       'private_shop_4', 'game_money_change_1', 'game_money_change_2',
       'game_money_change_3', 'game_money_change_4', 'enchant_count_1',
       'enchant_count_2', 'enchant_count_3', 'enchant_count_4',
       'combat_pledge_cnt_1', 'combat_pledge_cnt_2',
       'combat

In [84]:
X_train.head()

,acc_id,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,...,target_type1_week3_etc,target_type1_week4_etc,source_weapon,target_weapon,source_armor,target_armor,source_accessory,target_accessory,source_spell,target_spell
0,2.0,0,22.919696,22.980553,22.659883,22.973531,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0,0,1,0,0,0,0,0
1,5.0,15,0.000000,0.589845,0.213000,5.313287,0.000000,0.310229,0.162887,2.763336,...,0.000000e+00,0.000000,0,0,0,0,0,0,0,0
2,8.0,16,4.987936,4.889629,4.653223,4.267014,26.808113,1.325063,1.444694,1.594739,...,0.000000e+00,0.000000,0,0,0,0,0,0,1,0
3,17.0,16,5.406914,9.048973,7.831832,6.176989,25.931158,1.328780,1.362575,1.834339,...,7.190952e-08,0.000029,1,1,0,1,1,1,0,1
4,21.0,16,6.048253,7.363701,4.966870,3.452466,4.379366,4.602406,0.654252,1.135141,...,0.000000e+00,0.000000,1,0,0,1,0,1,0,0


In [65]:
# X_train.to_csv('X_train.csv', index = False)

In [83]:
y_train = train['leave']

In [85]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500], 'max_features' : [13, 14, 15, 16]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'accuracy', verbose = 2, n_jobs = -1)
grid_search.fit(X_train.iloc[:, 1:], y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 12.3min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [300, 500], 'max_features': [13, 14, 15, 16]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=2)

In [86]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7739333333333334
{'max_features': 16, 'n_estimators': 500}


In [87]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state = 42, max_features = 16, n_estimators = 500)
rf_clf.fit(X_train.iloc[:, 1:], y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=16, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [88]:
Importance = pd.DataFrame({'feature' : X_train.columns[1:], 'importance' : rf_clf.feature_importances_}).sort_values(by = 'importance', ascending = True)
Importance

,feature,importance
159,server_br,0.000014
157,server_bl,0.000030
156,server_bk,0.000032
155,server_bj,0.000039
49,enchant_count_1,0.000051
50,enchant_count_2,0.000060
158,server_bp,0.000072
52,enchant_count_4,0.000132
51,enchant_count_3,0.000174
152,server_bb,0.000316


In [59]:
y_pred = rf_clf.predict(X_test.loc[:, X_train.columns.values[1:]])
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [62]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test['leave'], y_pred)

0.7783

In [63]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test['leave'], y_pred)

array([[4580,  919],
       [1298, 3203]], dtype=int64)